In [1]:
%pylab inline

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from mpl_toolkits.mplot3d import axes3d

Populating the interactive namespace from numpy and matplotlib


In [ ]:
a,b,c = 1

In [2]:
import os
from time import time
from itertools import product, cycle

In [3]:
import ImarisLib

In [4]:
from cvbi.base_imaris.objects import GetSurpassObjects
from cvbi.base_imaris.stats import get_imaris_statistics

In [5]:
from sklearn.preprocessing import minmax_scale

In [6]:
def GetObjectId():
    
    
    vImarisLib = ImarisLib.ImarisLib()
    vServer = vImarisLib.GetServer()
    vNumberOfObjects = vServer.GetNumberOfObjects()
    
    for vIndex in range(vNumberOfObjects):
        vObjectId = vServer.GetObjectID(vIndex)
        return vObjectId; # work with the ID (return first one)
    
    return -1 # invalid id

In [7]:
from sklearn.externals import joblib
from sklearn.cluster import DBSCAN


def dbscan_predict(model, X):
    """
    Predict using dbscan

    :param model: trained dbscan object model from scikit learn
    :param X: New dataset to predict for, the shape should be the same as training data
    :return: numpy vector of predicted class
    """
    nr_samples = X.shape[0]

    y_new = np.ones(shape=nr_samples, dtype=int) * -1

    for i in range(nr_samples):
        diff = model.components_ - X[i, :]  # NumPy broadcasting

        dist = np.linalg.norm(diff, axis=1)  # Euclidean distance

        shortest_dist_idx = np.argmin(dist)

        if dist[shortest_dist_idx] < model.eps:
            y_new[i] = model.labels_[model.core_sample_indices_[shortest_dist_idx]]

    return(y_new)

In [8]:
aImarisId = GetObjectId()
vImarisLib = ImarisLib.ImarisLib()
vImaris = vImarisLib.GetApplication(aImarisId)
vDataSet = vImaris.GetDataSet()

In [9]:
imaris_file = vImaris.GetCurrentFileName()
imaris_dir  = os.path.dirname(imaris_file)
imaris_name = os.path.basename(imaris_file)

imaris_file,imaris_dir,imaris_name

('D:\\Box Sync\\Deb_Hen_Nilesh\\surfaces\\WT Th1 in OVA\\111717\\111717 REX3 CFA-OVA d5 OT-II Th1_f1_02_.ims',
 'D:\\Box Sync\\Deb_Hen_Nilesh\\surfaces\\WT Th1 in OVA\\111717',
 '111717 REX3 CFA-OVA d5 OT-II Th1_f1_02_.ims')

## Get Volume Data

In [10]:
nX = vDataSet.GetSizeX()
nY = vDataSet.GetSizeY()
nZ = vDataSet.GetSizeZ()
nC = vDataSet.GetSizeC()
nT = vDataSet.GetSizeT()

In [11]:
model_path = 'D:/Box Sync/Deb_Hen_Nilesh/surfaces/WT Th1 in OVA/111717/stats/111717 REX3 CFA-OVA d5 OT-II Th1_f1_02_.ims_CXCL10 cells_model.joblib'
db = joblib.load(model_path)

C:\Users\npatil4\Anaconda\envs\python27\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator DBSCAN from version 0.20.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [12]:
x_min,x_max = vDataSet.GetExtendMinX(),vDataSet.GetExtendMaxX()
y_min,y_max = vDataSet.GetExtendMinY(),vDataSet.GetExtendMaxY()
z_min,z_max = vDataSet.GetExtendMinZ(),vDataSet.GetExtendMaxZ()

In [13]:
x = np.arange(nX)
y = np.arange(nY)
z = np.arange(nZ)

X_all = np.array([[xi,yi,zi] for xi,yi,zi in product(x,y,z)])
X = X_all.copy()

In [14]:
Xs_index = X[:,0]
Ys_index = X[:,1]
Zs_index = X[:,2]

Xs_coords = minmax_scale(Xs_index.astype(np.float64),feature_range=(x_min,x_max))
Ys_coords = minmax_scale(Ys_index.astype(np.float64),feature_range=(y_min,y_max))
Zs_coords = minmax_scale(Zs_index.astype(np.float64),feature_range=(z_min,z_max))
pos_data = np.array([Xs_coords, Ys_coords, Zs_coords]).T

In [15]:
labels = dbscan_predict(db, X)

In [18]:
unique_labels,unique_label_counts = np.unique(ar=labels, return_counts=True)

colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
subset = labels!=-1

fig = plt.figure(figsize=(8,8),dpi=200)
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.grid(False); 
ax.xaxis.pane.fill =  ax.yaxis.pane.fill =  ax.zaxis.pane.fill = False
ax.scatter3D(xs = X[subset,0],
             ys = X[subset,1],
             zs = X[subset,2], 
             c = [colors[label] for label in labels[subset]],
             alpha=0.09, lw=0, s=10);

ax.set_xlim3d(0, 512)
ax.set_ylim3d(0, 512)
#ax.set_zlim3d(0, 60)
ax.view_init(elev=75, azim=-90);

#### Label one timepoint

In [28]:
data_list = vDataSet.GetDataVolumeFloats(aIndexC=1, aIndexT=0)
data_array = np.array(data_list)

data_out = np.zeros_like(data_array)

for i in tqdm_notebook(range(X.shape[0])):
    xi,yi,zi = X[i]
    
    if labels[i]!=-1:
        data_out[xi,yi,zi] = data_array[xi,yi,zi]
    
data_out_list = data_out.tolist()

In [16]:
vDataSet.SetSizeC(nC+1)

In [17]:
for ti in tqdm_notebook(range(nT)):
    
    data_list = vDataSet.GetDataVolumeFloats(aIndexC=1, aIndexT=ti)
    data_array = np.array(data_list)
    data_out = np.zeros_like(data_array)
    
    for i in tqdm_notebook(range(X.shape[0])):
        xi,yi,zi = X[i]
        if labels[i]!=-1:
            data_out[xi,yi,zi] = data_array[xi,yi,zi]

    data_out_list = data_out.tolist()
    vDataSet.SetDataVolumeFloats(aData=data_out_list, aIndexC=nC, aIndexT=ti)